In [1]:
# %matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib import gridspec
from scipy import stats, signal
from scipy.signal import hilbert
from numpy.lib.recfunctions import append_fields, merge_arrays
from nilearn import plotting
from pycircstat import mean as circmean
import pycircstat
from matplotlib.backends.backend_pdf import PdfPages

import sys
import pickle
import glob

from tarjan import tarjan
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.linear_model import LinearRegression as LR
import matplotlib.colors as clrs
import matplotlib.cm as cmx
import nilearn.plotting as ni_plot
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pylab as pyl
from PIL import Image
import os
from scipy.spatial.distance import pdist, squareform
from SubjectLevel.par_funcs_fine import *
from scipy.io import loadmat

from ptsa.data.readers import EEGReader
from ptsa.data.filters import ResampleFilter

from joblib import Parallel, delayed
import multiprocessing
import cluster_helper.cluster
import time
from collections import defaultdict
from types import SimpleNamespace

/home1/anup.das/.conda/envs/py3_env/lib/python3.7/site-packages/ptsa/data/readers/index.py:39: FutureWarning: Lab-specific readers may be moved to the cmlreaders package (https://github.com/pennmem/cmlreaders)
  FutureWarning)


In [2]:
# get TH1 subjects
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

subjs=RAM_helpers.get_subjs_and_montages('RAM_TH1')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

#Find subjects with only grids electrodes
Count_Grids = np.empty(0)
for i in range(0,np.shape(subjs)[0],1):
    subject = subjs['subject'].iloc[i]
    montage = subjs['montage'].iloc[i]
    monopol_chans = RAM_helpers.load_elec_info(subject, montage, bipolar=False)
    monopol_chans_types = monopol_chans['type']
    monopol_chans_types_counts_grids = monopol_chans_types.str.count('G')
    monopol_chans_types_counts_grids = sum(np.asarray(monopol_chans_types_counts_grids))
    Count_Grids = np.append(Count_Grids, np.array([monopol_chans_types_counts_grids]), axis=0)
    del subject
    del montage
    del monopol_chans
    del monopol_chans_types
    del monopol_chans_types_counts_grids
    
subjs_grids = subjs.iloc[np.nonzero(Count_Grids)]
subjs_grids = subjs_grids.reset_index(drop=True)
del subjs
th1subjs = subjs_grids
th1subjs['task'] = 'RAM_TH1'
th1subjs

,subject,montage,task
0,R1076D,0,RAM_TH1
1,R1147P,0,RAM_TH1
2,R1154D,0,RAM_TH1
3,R1155D,0,RAM_TH1
4,R1156D,0,RAM_TH1
5,R1167M,0,RAM_TH1
6,R1184M,0,RAM_TH1
7,R1190P,0,RAM_TH1
8,R1201P,0,RAM_TH1
9,R1202M,0,RAM_TH1


In [3]:
#Find minimum sampling frequency
Samp_Freq_All = np.empty(0)
for i in range(0,np.shape(th1subjs)[0],1):
    subject = th1subjs['subject'].iloc[i]
    montage = th1subjs['montage'].iloc[i]
    events = RAM_helpers.load_subj_events('RAM_TH1', subject, montage, as_df=True, remove_no_eeg=True)
    events_short = events.iloc[[0]]
    monopol_chans = RAM_helpers.load_elec_info(subject, montage, bipolar=False)
    eeg = RAM_helpers.load_eeg(events_short, rel_start_ms=0, rel_stop_ms=1500, buf_ms=1000, elec_scheme=monopol_chans, noise_freq=[58., 62.], resample_freq=None, pass_band=None, use_mirror_buf=False, demean=True, do_average_ref=True)
    Samp_Freq_All = np.append(Samp_Freq_All, np.array([float(eeg.samplerate)]), axis=0)
    del subject
    del montage
    del events
    del events_short
    del monopol_chans
    del eeg
    
Samp_Freq_All
np.unique(Samp_Freq_All)
Samp_Freq = np.min(np.unique(Samp_Freq_All))
Samp_Freq_All

array([1000., 1000., 1000., 1600., 1000., 1000., 1000., 1000.,  500.,
       1000., 1000., 1000., 1000., 1000.])

In [ ]:
for i_subj in range(0, len(th1subjs), 1): 
    
    subject = th1subjs['subject'].iloc[i_subj]
    montage = th1subjs['montage'].iloc[i_subj]
    events_tmp = RAM_helpers.load_subj_events('RAM_TH1', subject, montage, as_df=True, remove_no_eeg=True)
    events_short = events_tmp.iloc[[0]]
    monopol_chans = RAM_helpers.load_elec_info(subject, montage, bipolar=False)
    eeg = RAM_helpers.load_eeg(events_short, rel_start_ms=0, rel_stop_ms=1500, buf_ms=1000, elec_scheme=monopol_chans, noise_freq=[58., 62.], resample_freq=None, pass_band=None, use_mirror_buf=False, demean=True, do_average_ref=True)
    fs = np.array([float(eeg.samplerate)])

    dd1 = defaultdict(list)
    dd2 = defaultdict(list)

    # load events, find session available
    json_events = RAM_helpers_New.load_subj_events_T('RAM_TH1', subject, '0', use_json=True)
    sessions_avail = np.unique(json_events.session)


    for sesh in sessions_avail:
        sess_events = np.where(json_events.session == sesh)
        sess_idx = sess_events[0][0]
        eegfile = json_events[sess_idx].eegfile
        log_file = eegfile.split('ephys')[0] + 'behavioral/current_source/logs/session_log.txt'
        eeg_events = json_events[json_events.eegfile==eegfile]
        log = open(log_file, 'r')

        strs_START = ['TRIAL_NAVIGATION_STARTED']
        strs_END = ['TRIAL_NAVIGATION_ENDED']

        ms_times_start = []
        ms_times_end = []
        item_names = []
        for line in log.readlines():
            line = line.replace('\r','')
            tokens = line[:-1].split('\t')
            if len(tokens) > 1:
                if tokens[3] in strs_START:
                    ms_times_start.append(float(tokens[0]))
                if tokens[3] in strs_END:
                    ms_times_end.append(float(tokens[0]))

        #start
        events_ms_times = eeg_events.mstime
        events_eeg_offsets = eeg_events.eegoffset
        lr=LR(fit_intercept=True)
        lr.fit(X=events_ms_times.reshape(-1, 1), y=events_eeg_offsets.reshape(-1, 1))    
        target_offsets_start = lr.predict(np.array(ms_times_start).reshape(-1,1))
        target_offsets_end = lr.predict(np.array(ms_times_end).reshape(-1,1))

        for trial, offset in enumerate(target_offsets_start):
            # CREATE DATAFRAME
            info_for_df1 = {'subject': subject,
                           'eegoffset': int(target_offsets_start[trial]),
                           'mstime': ms_times_start[trial],
                           'type': 'TRIAL_NAVIGATION_STARTED',
                           'eegfile': eegfile,
                           'experiment': 'TH1',
                           'session': sesh}

            info_for_df2 = {'subject': subject,
                           'eegoffset': int(target_offsets_end[trial]),
                           'mstime': ms_times_end[trial],
                           'type': 'TRIAL_NAVIGATION_ENDED',
                           'eegfile': eegfile,
                           'experiment': 'TH1',
                           'session': sesh}

            for key, value in info_for_df1.items():
                dd1[key].append(value)
            for key, value in info_for_df2.items():
                dd2[key].append(value)
            df1 = pd.DataFrame(data = dd1)
            df2 = pd.DataFrame(data = dd2)



    self = SimpleNamespace()
    # whether to load bipolar pairs of electrodes or monopolar contacts
    self.bipolar = False

    # This will load eeg and compute the average reference before computing power. Recommended if bipolar = False.
    self.mono_avg_ref = True

    # power computation settings
    #self.start_time = 0
    self.start_time = 0
    self.end_time = 1500
    self.wave_num = 5
    self.buf_ms = 1000
    self.noise_freq = [58., 62.]
    #self.resample_freq = None
    #Anup changed here
    self.resample_freq = 500.
    self.log_power = True
    self.freqs = np.logspace(np.log10(3), np.log10(40), 200)
    self.mean_over_time = True
    self.time_bins = None
    self.use_mirror_buf = False
    self.pool = None

    # load electrode info
    self.subject = subject
    self.montage = montage
    self.elec_info = RAM_helpers.load_elec_info(self.subject, self.montage, self.bipolar)

    arr_tmp = np.array(df1['eegoffset']) + np.round((np.array(df2['eegoffset']) - np.array(df1['eegoffset']))/2) - (fs/1000)*750
    arr_int = arr_tmp.astype(int)
    events_for_computation = df1
    events_for_computation['eegoffset'] = arr_int
    events_for_computation['type'] = 'TRIAL_NAVIGATION'

    # compute power with RAM_helper function
    subject_data = RAM_helpers.compute_power(events_for_computation,
                                             self.freqs,
                                             self.wave_num,
                                             self.start_time,
                                             self.end_time,
                                             buf_ms=self.buf_ms,
                                             cluster_pool=self.pool,
                                             log_power=self.log_power,
                                             noise_freq=self.noise_freq,
                                             elec_scheme=self.elec_info,
                                             resample_freq=self.resample_freq,
                                             do_average_ref=self.mono_avg_ref,
                                             mean_over_time=self.mean_over_time,
                                             use_mirror_buf=self.use_mirror_buf,
                                             loop_over_chans=True)


    # default frequency settings for identifying peaks
    self.freqs = np.logspace(np.log10(3), np.log10(40), 200)
    self.bipolar = False
    self.start_time = 0
    self.end_time = 1500
    self.mono_avg_ref = True
    self.hilbert_start=0
    self.hilbert_end=1500

    # window size to find clusters (in Hz)
    self.cluster_freq_range = 2.

    # D: depths, G: grids, S: strips
    self.elec_types_allowed = ['G']

    # spatial distance considered near (mm)
    self.min_elec_dist = 25.

    # If True, osciallation clusters can't cross hemispheres
    self.separate_hemis = True

    # number of electrodes needed to be considered a cluster
    self.min_num_elecs = 6

    # elec_info column from which to extract x,y,z coordinates
    self.elec_pos_column = ''


    xyz = self.elec_info[['{}{}'.format(self.elec_pos_column, coord) for coord in ['tal.x', 'tal.y', 'tal.z']]].values
    if self.separate_hemis:
        if 'Loc1' in self.elec_info:
            xyz[self.elec_info['Loc1']=='Left Cerebrum', 0] -= 100
        else:
            xyz[xyz[:, 0] < 0, 0] -= 100
    elec_dists = squareform(pdist(xyz))

    # figure out which pairs of electrodes are closer than the threshold
    near_adj_matr = (elec_dists < self.min_elec_dist) & (elec_dists > 0.)
    allowed_elecs = np.array([e in self.elec_types_allowed for e in self.elec_info['type']])













    # default frequency settings for identifying peaks
    self.freqs = np.logspace(np.log10(3), np.log10(40), 200)
    self.bipolar = False
    self.start_time = 0
    self.end_time = 1500
    self.mono_avg_ref = True
    self.hilbert_start=0
    self.hilbert_end=1500

    # window size to find clusters (in Hz)
    self.cluster_freq_range = 2.

    # D: depths, G: grids, S: strips
    self.elec_types_allowed = ['G']

    # spatial distance considered near (mm)
    self.min_elec_dist = 25.

    # If True, osciallation clusters can't cross hemispheres
    self.separate_hemis = True

    # number of electrodes needed to be considered a cluster
    self.min_num_elecs = 6

    # elec_info column from which to extract x,y,z coordinates
    self.elec_pos_column = ''
    self.elec_pos_type = ''


    xyz = self.elec_info[['{}{}'.format(self.elec_pos_column, coord) for coord in ['tal.x', 'tal.y', 'tal.z']]].values
    if self.separate_hemis:
        if 'Loc1' in self.elec_info:
            xyz[self.elec_info['Loc1']=='Left Cerebrum', 0] -= 100
        else:
            xyz[xyz[:, 0] < 0, 0] -= 100
    elec_dists = squareform(pdist(xyz))

    # figure out which pairs of electrodes are closer than the threshold
    near_adj_matr = (elec_dists < self.min_elec_dist) & (elec_dists > 0.)
    allowed_elecs = np.array([e in self.elec_types_allowed for e in self.elec_info['type']])

    # normalized power spectra
    event_dim_str='event'
    self.subject_data = subject_data
    sessions = self.subject_data[event_dim_str].data['session']
    norm_spectra = np.empty(self.subject_data.shape)
    uniq_sessions = np.unique(sessions)
    for sess in uniq_sessions:
        sess_inds = sessions == sess

        m = np.mean(self.subject_data[sess_inds], axis=1)
        m = np.mean(m, axis=0)
        s = np.std(self.subject_data[sess_inds], axis=1)
        s = np.mean(s, axis=0)
        norm_spectra[sess_inds] = (self.subject_data[sess_inds] - m) / s
    p_spect = norm_spectra

    # Compute mean power spectra across events, and then find where each electrode has peaks
    mean_p_spect = np.mean(p_spect, axis=self.subject_data.get_axis_num('event'))
    peaks = par_find_peaks_by_chan2(mean_p_spect, self.freqs, 1)
    # now that we know at which each electrode has peaks, compute clusters of electrodes that exhibit peaks at
    # similar frequencies and are close enough together
    steplength=5;windowLength=15;
    distance=25;
    CLUSTERS=[]
    peaks[:, ~allowed_elecs] = False
    peakid=peaks
    windows=np.stack([[i,windowLength+i] for i in range(0,201-steplength,steplength)])
    for ire in range(10):
        peak_counts=np.sum(peakid,axis=1)
        window_counts=np.array([sum(peak_counts[w[0]:w[1]]) for w in windows])
        peak_window=np.argmax(window_counts)
        near_this_ev = near_adj_matr.copy()
        peak_within_window=np.any(peakid[windows[peak_window,0]:windows[peak_window,1]],axis=0)
        near_this_ev[~peak_within_window,:] = False
        near_this_ev[:, ~peak_within_window] = False
        # use targan algorithm to find the clusters
        graph = {}
        for elec, row in enumerate(near_this_ev):
            graph[elec] = np.where(row)[0]
        groups = tarjan(graph)
        #choose the connected componet with most electrodes as seed
        clusterSeed=sorted(sorted(groups,key=lambda a:-len(a))[0])
        #make it a dictionary
        window_true=np.zeros((200),dtype=bool)
        window_true[windows[peak_window,0]:windows[peak_window,1]]=True
        cluster={}
        if len(clusterSeed)>1:
            for i in clusterSeed:
                peak_freq=np.squeeze(np.where(np.logical_and(peakid[:,i],window_true))[0][0])
                cluster[i]=peak_freq
                peakid[peak_freq,i]=False
        if len(cluster)>1:
            for ire2 in range(10):
                for i in range(len(near_adj_matr)):
                    if i not in cluster:
                        near_freqS=np.squeeze(list(cluster.values()))[near_adj_matr[i,list(cluster.keys())]]
                        if len(near_freqS)>1:
                            window_true=np.zeros((200),dtype=bool)
                            window_true[windows[peak_window,0]:windows[peak_window,1]]=True
                            near_freq=int(np.median(near_freqS))
                            electrode_frequency=np.where(peakid[:,i])[0]
                            if np.any(np.abs(electrode_frequency-near_freq)<15):
                                peak_freq=np.array(min(electrode_frequency, key=lambda x:abs(x-near_freq)))
                                cluster[i]=peak_freq
                                peakid[peak_freq,i]=False
            CLUSTERS.append(cluster)
    for i in CLUSTERS:
        for j in i:
            i[j]=self.freqs[i[j]]
    res={}
    i=0
    while i<(len(CLUSTERS)):
        if len(CLUSTERS[i])>3:
            res[i]=CLUSTERS[i]
        i+=1
    cluster_count = 0
    df_list = []

    for i in res.keys():
        cluster_count += 1
        col_name = 'cluster{}'.format(cluster_count)
        cluster_df = pd.DataFrame(data=np.full(shape=(peaks.shape[1]), fill_value=np.nan), columns=[col_name])
        for j in res[i]:
            cluster_df.iloc[j]=res[i][j]
        df_list.append(cluster_df)
    df = None
    if df_list:
        df = pd.concat(df_list, axis='columns')
        df = pd.concat([df,self.elec_info], axis='columns')
        # add a tag to localization. avg.region for new patients, Loc3 for old
        if 'avg.region' in self.elec_info:
            df['tag'] = self.elec_info['avg.region']
        else:
            df['tag'] = self.elec_info['Loc3']

    self.res = {}        
    self.res['clusters'] = df













    events = events_for_computation
    self.task = th1subjs.iloc[0][2]
    Resample = Samp_Freq
    task_phase = 'TRIAL_NAVIGATION'

    # initialize eeg and res
    uniq_sessions = np.unique(self.subject_data.event.data['session'])
    if self.subject[:2]=='FR':
        noise=[48., 52.]
    else:
        noise=[58., 62.]
    buf=1000

    eeg=RAM_helpers.load_eeg(events, self.hilbert_start, self.hilbert_end, buf_ms=buf, elec_scheme=self.elec_info, noise_freq=noise, resample_freq=None, pass_band=None, use_mirror_buf=False, demean=True, do_average_ref=True)
    eeg = ResampleFilter(eeg, Resample).filter()

    print('Band pass EEG')
    time_frame_start=int((buf/1000)*Resample)
    time_frame_end=int(Resample*(self.hilbert_end-self.hilbert_start+buf)/1000)
    eeg=eeg.transpose('channel', 'event', 'time')
    thetas = np.radians(np.arange(0, 360, 5))
    rs = np.radians(np.arange(0, 18, 1))
    theta_r = np.stack([(x, y) for x in thetas for y in rs])
    params = np.stack([theta_r[:, 1] * np.cos(theta_r[:, 0]), theta_r[:, 1] * np.sin(theta_r[:, 0])], -1)
    f=circ_lin_regress
    allowed_elecs = np.array([e in self.elec_types_allowed for e in self.elec_info['type']])
    i=1
    print('Traveling wave analysis')
    while 'cluster{}'.format(i) in self.res['clusters'].columns:
        res={}
        clusterphase=[]
        clusterpower=[]
        clustereeg=[]
        cluster='cluster{}'.format(i)
        for j in range(len(self.res['clusters'])):
            freq=self.res['clusters'][cluster][j]
            if ~np.isnan(freq):
                feeg=RAM_helpers.band_pass_eeg(eeg[j], [freq *.85, freq/.85],order=3)
                clusterphase.append(np.angle(hilbert(feeg, N=feeg.shape[-1], axis=-1)))
                clusterpower.append(np.abs(hilbert(feeg, N=feeg.shape[-1], axis=-1)))
                clustereeg.append(feeg)
        res['power']=np.stack(clusterpower)[:,:,time_frame_start:time_frame_end].astype('float32') 
        res['phase']=np.stack(clusterphase)[:,:,time_frame_start:time_frame_end].astype('float32') 
        res['eeg']=np.stack(clustereeg)[:,:,time_frame_start:time_frame_end].astype('float32') 
        
        xyz_cluster = self.elec_info[['{}{}'.format(self.elec_pos_column, coord) for coord in ['tal.x', 'tal.y', 'tal.z']]].values
        xyz_cluster = xyz_cluster[~np.isnan(self.res['clusters'][cluster])]
        xyz_all_grid=self.elec_info[['{}{}'.format(self.elec_pos_column, coord) for coord in ['tal.x', 'tal.y', 'tal.z']]].values[np.squeeze(self.elec_info[['{}{}'.format(self.elec_pos_type, elec_type) for elec_type in ['type']]].values == 'G'), :]
        pca = PCA(n_components=3)

        xyz_all_grid -= np.mean(xyz_all_grid, axis=0)
        xyz_cluster -= np.mean(xyz_cluster, axis=0)

        xyz_2D_map = pca.fit_transform(xyz_all_grid)[:, :2]
        xyz_PCA = pca.transform(xyz_cluster)[:, :2]
        
        elec_dists = squareform(pdist(xyz_PCA))
        near_adj_matr = (elec_dists < 25.)
        local_angle={}
        local_sf={}
        local_rs={}
        local_off={}
        for j in tqdm(range(0,len(xyz_cluster))):
            if sum(near_adj_matr[j])>3:
                norm_coords = xyz_PCA[near_adj_matr[j],:]
                num_iters = int(res['phase'][near_adj_matr[j]].T.shape[0])
                data_as_list = zip(res['phase'][near_adj_matr[j]].T,np.array([norm_coords]*num_iters), [theta_r]*num_iters, [params]*num_iters)
                res_as_list = Parallel(n_jobs=40, verbose=0)(delayed(f)(x[0], x[1], x[2], x[3]) for x in data_as_list)
                local_angle[j]=np.stack([x[0] for x in res_as_list], axis=0).astype('float32') 
                local_sf[j]=np.stack([x[1] for x in res_as_list], axis=0).astype('float32') 
                local_rs[j]=np.stack([x[2] for x in res_as_list], axis=0).astype('float32')
                local_off[j]=np.stack([x[3] for x in res_as_list], axis=0).astype('float32')  
        res['direction'] = local_angle 
        res['spatial_freuency']= local_sf
        res['rs']= local_rs
        res['offs']= local_off
        if cluster not in self.res:
            self.res[cluster]={}
        self.res[cluster][task_phase]=res
        i+=1

    with open('/home1/anup.das/Results/tw_th_files_grids_2D/tw_files/tw_nav_files/tw_subj_' + subject + '_grids_2D_fine_CPCAM.pkl', 'wb') as f:
        pickle.dump(self.res, f)

100%|██████████| 43/43 [00:23<00:00,  1.80it/s]


Band pass EEG
Traveling wave analysis


100%|██████████| 116/116 [05:02<00:00,  2.61s/it]


Band pass EEG
Traveling wave analysis


100%|██████████| 76/76 [05:32<00:00,  4.37s/it]


...

100%|██████████| 92/92 [03:16<00:00,  2.14s/it]


Band pass EEG
Traveling wave analysis


100%|██████████| 90/90 [05:17<00:00,  3.53s/it]


Band pass EEG
Traveling wave analysis


 42%|████▏     | 8/19 [10:28<15:56, 86.92s/it]